> En este cuaderno recolecto datos de licencias ambientales a nivel nacional otorgadas por la *Dirección General de Medio Ambiente y Cambio Climático* y documentadas en el [Sistema Nacional de Información Ambiental](http://snia.mmaya.gob.bo/web/licenciamiento.php). Este código corrió al final de la tarde del 26 de Octubre de 2022. 

## Funciones

In [226]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from IPython.display import display, Markdown, clear_output
import pyarrow.parquet as pq
import pyarrow as pa

def busqueda_licencias():
    """
    Descarga una tabla html de todas las licencias ambientales otorgadas
    por la Dirección General de Medio Ambiente y Cambio Climático
    """
    
    data = {
        'tipoLA': '14',
        'codLA': '',
        'aop': '',
        'emp': '',
        'dpto': '0',
        'sec': '0',
        'modSNIA': '1',
    }

    response = requests.post('http://snia.mmaya.gob.bo/web/ListarLAs.php',  data=data)
    return BeautifulSoup(response.text, 'html.parser')

def parse_row(row):
    """
    Interpreta cada fila en la tabla descargada
    """
    
    fields = row.select('td')
    licencia, nombre_proyecto, empresa, sector = [f.get_text() for f in fields[1:5]]
    proyecto_codigo, proyecto_tipo = [i for i in re.findall(r"(\d*)", fields[-1].select('a')[0]['href']) if i != '']
    return dict(proyecto_codigo=proyecto_codigo, proyecto_tipo=proyecto_tipo, nombre_proyecto=nombre_proyecto, sector=sector, empresa=empresa, licencia=licencia)

def parse_table(html):
    """
    Procesa todas las filas y devuelve un dataframe
    """
    
    return pd.DataFrame([parse_row(row) for row in html.select('tr')[1:]])

def get_docs(proyecto_codigo, proyecto_tipo):
    """
    Descarga la lista de documentos por proyecto
    """
    
    response = requests.get(f'http://snia.mmaya.gob.bo/web/detalleAOP.php?codProyecto={proyecto_codigo}&tipo={proyecto_tipo}')
    return BeautifulSoup(response.text, 'html.parser')

def parse_ubicacion(html, proyecto_codigo, proyecto_tipo):
    """
    Extrae la ubicación de cada proyecto de la tabla de documentos
    """
    
    u = []
    for row in html.select('#tb_ubicacion tr')[1:]:
        u.append({**{'proyecto_codigo':proyecto_codigo, 'proyecto_tipo': proyecto_tipo}, **{nombre:f.get_text() for nombre, f in zip(['departamento', 'provincia', 'municipio'], row.select('td'))}})
    return u

def parse_doc(row):
    """
    Interpreta propiedades en cada documento
    """
    
    enlace = None
    fields = row.select('td')
    documento_codigo, documento_nombre, fecha = [f.get_text() for f in fields[1:4]]
    enlaces = row.select('td a')
    if len(enlaces) > 0:
        codigo, id_doc, tipo_doc, tipo = [i for i in re.findall(r"(\d*)", enlaces[0]['href']) if i != '']
        enlace = f'http://snia.mmaya.gob.bo/web/datosLA.php?id_proy={codigo}&id_doc={id_doc}&tipo_doc={tipo_doc}&tipo={tipo}'
    return dict(documento_codigo=documento_codigo, documento_nombre=documento_nombre, fecha=fecha, enlace=enlace)

def parse_docs(html):
    """
    Procesa todos los documentos
    """
    
    return pd.DataFrame([parse_doc(row) for row in html.select('#tb_proyectos tr')[1:]])

## Proceso

Construye una tabla de todos los proyectos con licencias ambientales otorgadas a nivel nacional por la *Dirección General de Medio Ambiente y Cambio Climático*

In [196]:
html = busqueda_licencias()
proyectos = parse_table(html)

In [197]:
display(Markdown(f'Existen {df.shape[0]} registros de licencias ambientales'))

Existen 16959 registros de licencias ambientales

In [227]:
# Algunas variables útiles
n = proyectos.shape[0]
proyectos_ubicaciones = []
documentos = []

Complementa esta tabla con información de la ubicación de cada proyecto y construye una segunda tabla con todos los documentos presentados a la *Dirección General de Medio Ambiente y Cambio Climático*

In [229]:
for i, proyecto in proyectos.iterrows():
    clear_output(wait=True)
    display(Markdown(f'{i+1}/{n}'))
    
    docs = get_docs(proyecto['proyecto_codigo'], proyecto['proyecto_tipo'])
    
    proyecto_ubicaciones = parse_ubicacion(docs, proyecto['proyecto_codigo'], proyecto['proyecto_tipo'])
    proyectos_ubicaciones.append(proyecto_ubicaciones)
    
    d = parse_docs(docs)
    d.insert(0, 'proyecto_tipo', proyecto['proyecto_tipo'])
    d.insert(0, 'proyecto_codigo', proyecto['proyecto_codigo'])
    documentos.append(d)

16959/16959

Consolida toda la información

In [245]:
ubi = []
for u in proyectos_ubicaciones:
    ubi.extend(u)

ubicaciones_df = pd.DataFrame(ubi)
documentos_df = pd.concat(documentos)

## Resultados

Cada tabla incluye columnas que ayudan a cruzar información (`proyecto_codigo` y `proyecto_tipo`) entre sí. Una muestra de cada tabla:

**proyectos**

In [248]:
proyectos

,proyecto_codigo,proyecto_tipo,nombre_proyecto,sector,empresa,licencia
0,3,1,REDES DE TRANSMISION EN MEDIA Y BAJA TENSION E...,Energía,COOPERATIVA RURAL DE ELECTRIFICACION LTDA. - C...,...
1,5,1,PROYECTO PRESA DE REGULACION LANDARA,Energía,EMPRESA RIO ELECTRICO S.A.,...2-05-CD 3 No 2739/06
2,6,1,"PROYECTO: ELECTRIFICACION RURAL ""LLAJMAPAMPA""",Energía,PREFECTURA DEL DEPARTAMENTO DE LA PAZ,...1-05-CD 4 No 2740/06
3,7,1,"ELECTRIFICACION RURAL ""SAYTU ANTAQUE""",Energía,PREFECTURA DEL DEPARTAMENTO DE LA PAZ,...4-05-CD 4 No 2741/06
4,9,1,"ELECTRIFICACION RURAL ""VILLA PORVENIR""",Energía,PREFECTURA DEL DEPARTAMENTO DE LA PAZ,...4-05-CD 4 No 2743/06
...,...,...,...,...,...,...
16954,21604,2,SISTEMA DE DISTRIBUCION DE GAS NATURAL LOCALID...,Hidrocarburos,YACIMIENTOS PETROLIFEROS FISCALES BOLIVIANOS -...,...04/LASP/N° 475/2022
16955,21660,2,PROYECTO DE RIEGO ITANGUA CUEVO,Agropecuario,GOBIERO AUTONOMO DEPARTAMENTAL DE CHUQUISACA,...01/DAA/N° 5085/2022
16956,21660,2,PROYECTO DE RIEGO ITANGUA CUEVO,Agropecuario,GOBIERO AUTONOMO DEPARTAMENTAL DE CHUQUISACA,...1/LASP/N° 5085/2022
16957,21684,1,CONSTRUCCION TINGLADO MUNICIPIO CHOQUECOTA,Multisectorial,GOBIERNO AUTONOMO MUNICIPAL DE CHOQUECOTA,...22


**ubicaciones**

In [246]:
ubicaciones_df

,proyecto_codigo,proyecto_tipo,departamento,provincia,municipio
0,3,1,Santa Cruz,Chiquitos,
1,5,1,Potosí,Antonio Quijarro,Tomave
2,6,1,La Paz,Murillo,
3,7,1,La Paz,Murillo,
4,9,1,La Paz,Pacajes,Comanche
...,...,...,...,...,...
83665,21660,2,Santa Cruz,Cordillera,Cuevo
83666,21660,2,Chuquisaca,Luis Calvo,Huacaya
83667,21660,2,Santa Cruz,Cordillera,Cuevo
83668,21684,1,Oruro,Carangas,Choquecota


**documentos**

In [253]:
documentos_df.fecha = pd.to_datetime(documentos_df.fecha, format='%d/%m/%Y')

In [254]:
documentos_df

,proyecto_codigo,proyecto_tipo,documento_codigo,documento_nombre,fecha,enlace
0,3,1,FA-2175,Ficha Ambiental,2005-10-20,None
1,3,1,FA-2175a,Ficha Ambiental,2006-01-03,None
2,3,1,PPM-PASA-2175,Programa de Prevención y Mitigación-Plan de Ap...,2006-06-30,None
3,3,1,CD-2175,Certificado de Dispensación,2006-07-03,http://snia.mmaya.gob.bo/web/datosLA.php?id_pr...
0,5,1,FA-2739,Ficha Ambiental,2006-01-03,None
...,...,...,...,...,...,...
3,21660,2,LASP-5085,Licencia de Actividades con Sustancias Peligrosas,2022-10-10,http://snia.mmaya.gob.bo/web/datosLA.php?id_pr...
0,21684,1,EMA-5086,Solicitud de Certificado de Exención de MA,2022-10-03,None
1,21684,1,CEMA-5086,Certificado de Exención de MA,2022-10-10,http://snia.mmaya.gob.bo/web/datosLA.php?id_pr...
0,21685,1,EMA-5087,Solicitud de Certificado de Exención de MA,2022-10-03,None


In [256]:
proyectos[proyectos.empresa.str.lower().str.contains('cooperativa')]

,proyecto_codigo,proyecto_tipo,nombre_proyecto,sector,empresa,licencia
0,3,1,REDES DE TRANSMISION EN MEDIA Y BAJA TENSION E...,Energía,COOPERATIVA RURAL DE ELECTRIFICACION LTDA. - C...,...
61,69,1,MODIFICACION RED ELECTRICA CANAL LIBERTAD,Energía,COOPERATIVA RURAL DE ELECTRIFICACION LTDA. - C...,...-05-CD 3 Nº 2813/07
249,183,1,AMPLIACION PLANTA GENERADORA CRE- LAS MISIONES,Energía,COOPERATIVA RURAL DE ELECTRIFICACION LTDA. - C...,...5 - CD 3 N° 2675106
252,186,1,AMPLIACION PLANTA GENERADORA CRE-SAN IGNACIO D...,Energía,COOPERATIVA RURAL DE ELECTRIFICACION LTDA. - C...,...5 - CD 3 Nº 2662/06
282,218,1,PROYECTO TELEFONIA PUBLICA V-SAR FASE II COTAS...,Comunicaciones,COOPERATIVA DE TELECOMUNICACIONES SANTA CRUZ L...,...-07-CD 3-Nº 3267/08
...,...,...,...,...,...,...
16880,21138,2,EXPLOTACION Y CONCETRACION DE ORO ALUVIAL - AT...,Minero,COOPERATIVA MINERA AURIFERA ASOBAL MADRE DE DI...,...02/DAA/N° 5008/2022
16881,21139,2,EXPLOTACION Y CONCENTRACION DE ORO ALUVIAL - A...,Minero,COOPERATIVA MINERA AURIFERA ASOBAL - MADRE DE ...,...02/DAA/N° 5009/2022
16940,21463,2,EXTRACCION DE YESO DE LA ATE YURAJ LOMA HIGUERANI,Minero,COOPERATIVA MINERA YURAJ LOMA HIGUERANI R.L.,...02/DAA/N° 5063/2022
16942,21484,2,EXPLOTACIONA A CIELO ABIERTO Y CONCENTRACION D...,Minero,COOPERATIVA MINERA AURIFERA ORIGINARIOS PUERTO...,...02/DAA/N° 5065/2022


In [258]:
documentos_df.dtypes

proyecto_codigo             object
proyecto_tipo               object
documento_codigo            object
documento_nombre            object
fecha               datetime64[ns]
enlace                      object
dtype: object

Guardar todo en parquet

In [250]:
for name, dfi in zip(['proyectos', 'ubicaciones', 'documentos'], [proyectos, ubicaciones_df, documentos_df]):
    pq.write_table(pa.Table.from_pandas(dfi), f'data/{name}.parquet')

Guardar todo en csv

In [259]:
for name, dfi in zip(['proyectos', 'ubicaciones', 'documentos'], [proyectos, ubicaciones_df, documentos_df]):
    dfi.to_csv(f'data/{name}.csv', index=False)